In [58]:
import pandas as pd
from duckdb import query
import pandas as pd
import numpy as np
pd.options.display.max_columns = 200
pd.options.display.max_rows = 500
pd.set_option('mode.chained_assignment', None)

In [59]:
word = pd.read_pickle('../../pickles/word.pkl')
pars = pd.read_pickle('../../pickles/pars.pkl')
lexn = pd.read_pickle('../../pickles/lexn.pkl')
pdgm = pd.read_pickle('../../pickles/pdgm.pkl')
book = pd.read_pickle('../../pickles/book.pkl')
chap = pd.read_pickle('../../pickles/chap.pkl')
vers = pd.read_pickle('../../pickles/vers.pkl')
frlc = pd.read_pickle('../../pickles/frlc.pkl')
frlb = pd.read_pickle('../../pickles/frlb.pkl')
gnt = pd.read_pickle('../../pickles/gnt.pkl')

In [60]:
rows = []
for lexn_id in frlc.frlc_lexn_id.unique():
    temp = frlc[frlc.frlc_lexn_id==lexn_id]
    s = ""
    for bk,ch in list(zip(temp.frlc_book_name_abbrev,temp.frlc_chap_num)):
        s = s + "|" + bk + "_" + str(ch)
    row = {
        "lexn_id": lexn_id,
        "lexn_chs": s[1:],
        # "len": len(s)
    }
    rows.append(row)
temp = pd.DataFrame(rows)    
lexn = lexn.merge(temp, how='left', on='lexn_id')
lexn.to_pickle("../../pickles/lexn.pkl")

In [ ]:
df=word[word.word_id.between('19010101','19010499')]
df.head(10)[['word_greek','word_english','word_pars_id','word_lexn_id']]

### functions

In [ ]:
def back_to_back_words(w1, w2):
    x = False
    verses = []
    for i,id in enumerate(word.word_lexn_id):
        if id == w1:
            x = True
        elif x and id == w2:
            x = False
            verses.append(word.iloc[i].word_vers_id)
        else:
            x = False
    return verses

In [ ]:
def lexn_gloss_lookup(arr):
    return list(lexn[lexn.lexn_gloss.isin(arr)].lexn_id)


In [ ]:
def get_interlinear(vers_ids):
    verses = []
    for vers_id in vers_ids:
        vers = gnt[gnt.vers_id==vers_id]        
        vers_ref_abbrev = list(vers.vers_ref_abbrev.unique())[0]
        verses.append([vers_ref_abbrev,]+[(word).replace('¶','') for word,punc in list(zip(vers.word_greek, vers.word_greek_punc))])
        verses.append([vers_ref_abbrev,]+[(word).replace('¶','') for word,punc in list(zip(vers.word_english, vers.word_english_punc))])
    df = pd.DataFrame(verses).fillna('')
    df = df.rename(columns={n:f"word_{n}" for n in df.columns})
    df = df.rename(columns={"word_0":"vers_ref_abbrev"})
    return df

In [ ]:
lexn_gloss_lookup(['to talk', 'to speak', 'to say'])

In [ ]:
lexn_gloss_lookup(['God'])

In [ ]:
get_interlinear(vers_ids)

In [ ]:
vers_ids = back_to_back_words('0000', '0114')
vers_ids

In [ ]:
vers1 = [(word + punc).replace('¶','') for word,punc in list(zip(vers.word_greek, vers.word_greek_punc))]

In [ ]:
verses = [vers1, ]

In [ ]:
pd.DataFrame(verses).fillna('')